In [1]:
import json, pathlib, requests, pandas as pd

OUT = pathlib.Path("fpl_data")
OUT.mkdir(exist_ok=True)

# Endpoints (Classic + Draft)
CLASSIC_BOOTSTRAP = "https://fantasy.premierleague.com/api/bootstrap-static/"
DRAFT_BOOTSTRAP   = "https://draft.premierleague.com/api/bootstrap-static"
DRAFT_DYNAMIC     = "https://draft.premierleague.com/api/bootstrap-dynamic"

/Users/kirti8192/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:


def fetch_json(url):
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return r.json()


In [3]:

def write_json(obj, path):
    path.write_text(json.dumps(obj, indent=2))


In [4]:

def classic_elements_to_csv(data, path_csv):
    el = pd.DataFrame(data["elements"])
    teams = pd.DataFrame(data["teams"])[["id","name"]].rename(columns={"id":"team_id","name":"team_name"})
    pos = pd.DataFrame(data["element_types"])[["id","singular_name_short"]].rename(columns={"id":"element_type","singular_name_short":"pos"})
    el = el.merge(teams, left_on="team", right_on="team_id", how="left")
    el = el.merge(pos, left_on="element_type", right_on="element_type", how="left")
    # keep a useful subset
    keep_cols = [
        "id","first_name","second_name","web_name","team_name","pos",
        "now_cost","selected_by_percent","status","chance_of_playing_next_round",
        "total_points","minutes","goals_scored","assists","clean_sheets",
        "creativity","influence","threat","ict_index"
    ]
    el[keep_cols].to_csv(path_csv, index=False)


In [5]:

def draft_elements_to_csv(static_data, dynamic_data, path_csv):
    # Draft endpoints also have 'elements'
    el = pd.DataFrame(static_data["elements"])
    # dynamic contains live availability / statuses
    # (merge on 'id' if you want additional fields)
    el.to_csv(path_csv, index=False)


In [6]:

# Classic
classic = fetch_json(CLASSIC_BOOTSTRAP)


In [7]:
write_json(classic, OUT/"classic_bootstrap.json")


In [8]:
classic_elements_to_csv(classic, OUT/"classic_players.csv")


In [10]:
# Draft
draft_static  = fetch_json(DRAFT_BOOTSTRAP)

In [11]:
draft_dynamic = fetch_json(DRAFT_DYNAMIC)

In [12]:
write_json(draft_static,  OUT/"draft_bootstrap.json")
write_json(draft_dynamic, OUT/"draft_dynamic.json")

In [13]:
draft_elements_to_csv(draft_static, draft_dynamic, OUT/"draft_players_raw.csv")

In [14]:
print("Saved:")
for p in OUT.iterdir():
    print(" -", p)

Saved:
 - fpl_data/classic_players.csv
 - fpl_data/draft_dynamic.json
 - fpl_data/draft_players_raw.csv
 - fpl_data/classic_bootstrap.json
 - fpl_data/draft_bootstrap.json
